<a href="https://colab.research.google.com/github/efrizac/ABSA-using-SVM-on-User-Feedback-of-Digital-Signature-and-E-Stamp-Applications/blob/main/Aspect_Based_Sentiment_Analysis_Classification_using_SVM_on_User_Feedback_of_Digital_Signature_and_E_Stamp_Applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Scraping

In [ ]:
import pandas as pd
import numpy as np
import json, os, uuid

In [ ]:
!pip install app_store_scraper
from app_store_scraper import AppStore

In [ ]:
!pip install google-play-scraper
from google_play_scraper import app

##Scraping Google Play Store

#### All Review

In [ ]:
!pip install google-play-scraper
from google_play_scraper import app

from google_play_scraper import Sort, reviews_all

g_reviews = reviews_all(
    #'id.vida.app',
    #'id.co.xignature',
    'com.privygate.privyid',
    sleep_milliseconds=0,
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
)

#### Certain Amount

In [ ]:
from google_play_scraper import Sort, reviews

g_reviews, continuation_token = reviews(
    # 'id.vida.app',
    #'id.co.xignature',
    'com.privygate.privyid',
    lang='id',
    country='id',
    sort=Sort.NEWEST,
    count=700,
    filter_score_with=None

###Save Data

In [ ]:
g_df = pd.DataFrame(np.array(g_reviews),columns=['review'])
g_df = g_df.join(pd.DataFrame(g_df.pop('review').tolist()))
g_df.head(20)

In [ ]:
len(g_df.index)

In [ ]:
g_df[['score','at', 'content']].head(20)

In [ ]:
g_df = g_df[['score','at', 'content']]
g_df.rename(columns={'score':'rating','content':'review'}, inplace=True)
g_df.insert(loc=0, column='source', value='Google Play Store')
g_df.insert(loc=1, column='app', value='Privy')
g_df = g_df.sort_values(by='at', ascending=False) #scraping sudah dilakukan berdasarkan data terbaru
g_df.head(20)

In [ ]:
g_df.to_csv('vida_newest_googleplay_reviews.csv', index = False)

In [ ]:
g_df.to_csv('xignature_newest_googleplay_reviews.csv', index = False)

In [ ]:
g_df.to_csv('privy_relevant_googleplay_reviews.csv', index = False)

##Scraping App Store

In [ ]:
!pip install app_store_scraper
from app_store_scraper import AppStore

#a_reviews = AppStore("id", "privy", "1218828609")
a_reviews = AppStore("id", "vida", "1661262113")


def scrape_reviews(total_reviews, batch_size=400):
    current_page = 1
    reviews = []

    while len(reviews) < total_reviews:
        a_reviews.review(how_many=batch_size, after=reviews[-1]['date'] if reviews else None)
        new_reviews = a_reviews.reviews

        if not new_reviews:
            break

        reviews.extend(new_reviews)
        print(f"Page {current_page} - Total Reviews Scraped: {len(reviews)}")

        current_page += 1

    return reviews

reviews_data = scrape_reviews(total_reviews=400, batch_size=400)

###Save Data

In [ ]:
a_df = pd.DataFrame(np.array(reviews_data), columns=['review'])
a_df = a_df.join(pd.DataFrame(a_df.pop('review').tolist()))
a_df.head(5)

In [ ]:
len(a_df.index)

In [ ]:
a_df[['rating','date', 'review']].head(20)

In [ ]:
a_df = a_df[['rating','date', 'review']]
a_df.rename(columns={'date':'at'}, inplace=True)
a_df.insert(loc=0, column='source', value='App Store')
a_df.insert(loc=1, column='app', value='Vida')
sorted_a_df = a_df.sort_values(by='at', ascending=False) #scraping sudah dilakukan berdasarkan data terbaru
sorted_a_df.head(20)

In [ ]:
sorted_a_df.to_csv('vida_appstore_reviews.csv', index = False)

In [ ]:
sorted_a_df.to_csv('privy_appstore_reviews.csv', index = False)

##Merge Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

privy_g_df = pd.read_csv('/content/drive/MyDrive/privy_newest_googleplay_reviews.csv')
vida_g_df = pd.read_csv('/content/drive/MyDrive/vida_newest_googleplay_reviews.csv')
xignature_g_df = pd.read_csv('/content/drive/MyDrive/xignature_newest_googleplay_reviews.csv')
vida_a_df = pd.read_csv('/content/drive/MyDrive/vida_appstore_reviews.csv')
privy_a_df = pd.read_csv('/content/drive/MyDrive/privy_appstore_reviews.csv')

In [ ]:
merged_df = pd.concat([privy_g_df, vida_g_df, xignature_g_df, vida_a_df, privy_a_df], ignore_index=True)
sorted_merged_df = merged_df.sort_values(by='at', ascending=False)
sorted_merged_df.to_csv('merged_reviews.csv', index = False)
sorted_merged_df.shape

#Data Exploration

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data_Skripsi/merged_reviews.csv')

In [ ]:
data.head()

In [ ]:
print(data.head(5))
print('----------')
print('Shape: ' , data.shape)
print('----------')
print(data.dtypes)
print('----------')
print(data.describe())
print('----------')
print('Data Null?')
print(data.isnull().sum())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

review_counts = data.groupby(['app', 'source'])['review'].count().reset_index()

plt.figure(figsize=(10, 6))
ax = sns.barplot(x='app', y='review', hue='source', data=review_counts)
plt.title('Jumlah Ulasan per Sumber and Aplikasi')
plt.xlabel('Aplikasi')
plt.ylabel('Jumlah Ulasan')

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                fontsize=10, color='black',
                xytext=(0, 5), textcoords='offset points')

plt.show()


In [ ]:
!pip install wordcloud

from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = " ".join(review for review in data.review)
print ("There are {} words in the combination of all review.".format(len(text)))
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)

plt.figure(figsize=(10, 5), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

#Data Preprocessing

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

##Eliminate Data Duplication

In [ ]:
def check_duplicate_rows(data) :
  duplicate_rows = data[data.duplicated(keep=False)]
  return duplicate_rows

duplicate_rows = check_duplicate_rows(data)

if not duplicate_rows.empty:
  print("Baris duplikat ditemukan:")
  print(duplicate_rows)
else:
  print("Tidak ditemukan baris duplikat.")

In [ ]:
def remove_duplication(data):
  data.drop_duplicates(subset=['review'], keep='first', inplace=True)
  return data

data = remove_duplication(data)
data.shape

##Eliminate Short Reviews

In [ ]:
def check_words(data):
  short_reviews = data[data['review'].str.split().str.len() < 3]
  return short_reviews

short_reviews = check_words(data)

if not short_reviews.empty:
  print("Baris dengan review kurang dari 3 kata:")
  print(short_reviews)
else:
  print("Tidak ditemukan baris dengan review kurang dari 3 kata.")

In [ ]:
def remove_short_reviews(data):
  data = data[data['review'].str.split().str.len() >= 3]
  return data

data = remove_short_reviews(data)
data.shape

##Text Cleaning & Case Folding

In [ ]:
def case_folding(text):
  text = text.lower()
  return text

In [ ]:
def clean_text(text):
  text = text.lower()  # Mengubah teks menjadi huruf kecil
  text = re.sub(r'[^a-zA-Z\s]', '', text)  # Menghapus karakter khusus dan angka
  text = re.sub(r'[^\w\s]', '', text) # Menghapus karakter non-alphanumeric dan whitespace
  text = re.sub(r'\s+', ' ', text).strip()  # Menghapus whitespace ekstra
  return text

data['cleaned_review'] = data['review'].apply(clean_text)

comparison_df = data[['source', 'app', 'review', 'cleaned_review']]
print(comparison_df.head(10))

comparison_df.to_csv('review_comparison.csv', index=False)

##Tokenization

In [ ]:
def tokenize_text(text):
  return word_tokenize(text)

In [ ]:
def tokenize_text(text):
  return word_tokenize(text)

data['tokenized_review'] = data['cleaned_review'].apply(tokenize_text)

comparison_tokenized_df = data[['source', 'app', 'review', 'cleaned_review', 'tokenized_review']]
print(comparison_tokenized_df.head(10))

comparison_tokenized_df.to_csv('tokenized_review_comparison.csv', index=False)

##Normalization

In [ ]:
normalizad_word = pd.read_csv('/content/drive/MyDrive/Data_Skripsi/kamusalay.csv', header=None)
normalizad_word_dict = {row[0]: row[1] for _, row in normalizad_word.iterrows()}

def normalize_term(tokens):
    return [normalizad_word_dict.get(term, term) for term in tokens]

In [ ]:
normalizad_word = pd.read_csv('/content/drive/MyDrive/Data_Skripsi/kamusalay.csv', header=None)  # Memuat kamus alay (file .csv kamus normalisasi)
normalizad_word_dict = {row[0]: row[1] for _, row in normalizad_word.iterrows()}  # Membuat kamus normalisasi dari csv

def normalize_term(tokens):
    return [normalizad_word_dict.get(term, term) for term in tokens]  # Jika term ada di kamus, normalisasikan, jika tidak tetap

data['normalized_review'] = data['tokenized_review'].apply(normalize_term)

comparison_normalization_df = data[['source', 'app', 'review', 'tokenized_review', 'normalized_review']]
print(comparison_normalization_df.head(10))


comparison_normalization_df.to_csv('normalization_comparison.csv', index=False)

##Stopword Removal

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('indonesian'))

stop_words.update(["aplikasi", "privy", "privyid", "vida", "xignature", "akulaku"])

def remove_stopwords(tokens):
  filtered_tokens = [token for token in tokens if token not in stop_words]
  return filtered_tokens

data['filtered_review'] = data['normalized_review'].apply(remove_stopwords)

comparison_stopwords_df = data[['source', 'app', 'review', 'normalized_review', 'filtered_review']]
print(comparison_stopwords_df.head(10))

comparison_stopwords_df.to_csv('stopwords_removal_comparison.csv', index=False)

##Stemming

In [ ]:
!pip install Sastrawi

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Membuat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming_text(tokens):
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  return stemmed_tokens

data['stemmed_review'] = data['filtered_review'].apply(stemming_text)

comparison_stemming_df = data[['source', 'app', 'review', 'filtered_review', 'stemmed_review']]
print(comparison_stemming_df.head(10))

comparison_stemming_df.to_csv('stemming_comparison.csv', index=False)

#Topic Modeling Using LDA (Latent Dirichlet Allocation)

In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.tokenize import word_tokenize
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/preprocessed_data.csv')
data

In [ ]:
dictionary = corpora.Dictionary(data['stemmed_review'])
corpus = [dictionary.doc2bow(text) for text in data['stemmed_review']]

coherence_values = []
perplexity_values = []
num_topics_range = range(2, 12)

for num_topics in num_topics_range:
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10, alpha='auto')
    coherence_model = CoherenceModel(model=lda_model, texts=data['stemmed_review'], dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherence_model.get_coherence())
    perplexity_values.append(lda_model.log_perplexity(corpus))

coherence_df = pd.DataFrame({'Jumlah Topik': num_topics_range, 'Nilai Koherensi': coherence_values})

In [ ]:
# Membuat model LDA
dictionary = corpora.Dictionary(data['stemmed_review'])
corpus = [dictionary.doc2bow(text) for text in data['stemmed_review']]
optimal_num_topics = 4 # Sesuaikan dengan jumlah topik yang optimal
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=optimal_num_topics, random_state=42, passes=10, alpha='auto')

# Menampilkan topik yang ditemukan
print("Topik yang ditemukan:")
topics = lda_model.print_topics(num_words=15)
for idx, topic in topics:
    print(f"Topik {idx}:\n{topic}\n")

print("Keyword per Topik")
topics = lda_model.print_topics(num_words=60)
for idx, topic in topics:
    print(f"Topik {idx}:\n{topic}\n")

In [ ]:
!pip install pyLDAvis

import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Visualisasi LDA dengan pyLDAvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

#Labeling Data

In [ ]:
import pandas as pd
from collections import Counter

##Aspect Labeling

In [ ]:
topic0 = [
   "daftar", "email", "masuk", "susah", "verifikasi", "login", "id", "tolong", "ya", "nik", "kali", "gagal", "akun", "ganti", "foto", "sih", "password", "ktp", "coba", "mohon", "data", "ulang", "swafoto", "menit", "liveness", "kamera", "register", "reg", "vermuk", "face"
]

topic1 = [
    "mudah", "bantu", "tanda", "tangan", "banget", "bagus", "digital", "ribet", "ttd", "dokumen", "kerja", "bikin", "aplikasi", "guna", "app", "keren", "aplikasi", "mantap", "recommend"
]

topic2 = [
    "respon", "cepat", "layan", "cs", "chat", "kasih", "kirim", "dm", "instagram", "terima", "tanggap", "balas", "live", "fast", "ramah", "solusi", "terimakasih", "live", "helpdesk", "admin", "service", "komplain"
]

topic3 = [
    "buka", "update", "pakai", "koneksi", "download", "jaring", "apk", "youtube", "eror", "error", "internet", "bug", "lot", "lelet", "baik", "instal", "install", "hitam", "lemot", "loading", "server", "lag", "upgrade", "store", "maintenance"
]

def label_aspects(stemmed_review):
    topic0_label = 1 if any(keyword in stemmed_review for keyword in topic0) else 0
    topic1_label = 1 if any(keyword in stemmed_review for keyword in topic1) else 0
    topic2_label = 1 if any(keyword in stemmed_review for keyword in topic2) else 0
    topic3_label = 1 if any(keyword in stemmed_review for keyword in topic3) else 0

    return pd.Series({
        'Login dan Verifikasi': topic0_label,
        'Efisiensi': topic1_label,
        'Layanan Pengguna': topic2_label,
        'Performa': topic3_label
    })

data[['Login dan Verifikasi', 'Efisiensi', 'Layanan Pengguna', 'Performa']] = data['stemmed_review'].apply(label_aspects)
data['total_aspek'] = data[['Login dan Verifikasi', 'Efisiensi', 'Layanan Pengguna', 'Performa']].sum(axis=1)

print(data.head())

##Sentiment Labeling

In [ ]:
#Mengubah angka 1 menjadi "Relevant"
columns_to_replace = ['Login dan Verifikasi (T)', 'Efisiensi (T)', 'Layanan Pengguna (T)', 'Responsivitas/Performa (T)']
for col in columns_to_replace:
    data.loc[pd.notnull(data[col]), col] = "Relevant"

data.to_csv('annotator_data.csv', index=False)
data

In [ ]:
from collections import Counter

ann1 = pd.read_csv('/content/drive/MyDrive/annotator_data_sentiment_1.csv')
ann2 = pd.read_csv('/content/drive/MyDrive/annotator_data_sentiment_2.csv')
ann3 = pd.read_csv('/content/drive/MyDrive/annotator_data_sentiment_3.csv')

merged_df = pd.DataFrame({
  'review' : ann1['review'],
  'stemmed_review' : ann1['stemmed_review'],
  'login_verifikasi_1' : ann1['Login dan Verifikasi (T)'],
  'login_verifikasi_2' : ann2['Login dan Verifikasi (T)'],
  'login_verifikasi_3' : ann3['Login dan Verifikasi (T)'],
  'efisiensi_1' : ann1['Efisiensi (T)'],
  'efisiensi_2' : ann2['Efisiensi (T)'],
  'efisiensi_3' : ann3['Efisiensi (T)'],
  'layanan_pengguna_1' : ann1['Layanan Pengguna (T)'],
  'layanan_pengguna_2' : ann2['Layanan Pengguna (T)'],
  'layanan_pengguna_3' : ann3['Layanan Pengguna (T)'],
  'responsivitas_1' : ann1['Responsivitas/Performa (T)'],
  'responsivitas_2' : ann2['Responsivitas/Performa (T)'],
  'responsivitas_3' : ann3['Responsivitas/Performa (T)'],
})

In [ ]:
import numpy as np

merged_df = merged_df.fillna('0')
merged_df = merged_df.replace('IR', '0')

# Membersihkan nilai dan menghilangkan desimal
def clean_value(x):
    if pd.isna(x):  # Jika nilai NaN, biarkan sebagai NaN
        return x
    try:
        x = float(x)  # Ubah ke float terlebih dahulu
        return int(x)  # Ubah ke integer untuk menghilangkan titik desimal
    except (ValueError, TypeError):
        return x

merged_df = merged_df.applymap(clean_value)

merged_df

In [ ]:
def get_final_sentiment(row, columns):
    # Hitung frekuensi setiap nilai di kolom yang diberikan
    sentiment_counts = Counter([row[col] for col in columns])
    most_common_sentiment = sentiment_counts.most_common(1)[0]

    # Jika mayoritas sentimen muncul dua kali atau lebih
    if most_common_sentiment[1] >= 2:
        return most_common_sentiment[0]
    return 'equal'

# Ambil nilai dominan untuk setiap aspek
merged_df['final_login_verifikasi'] = merged_df.apply(
    lambda row: get_final_sentiment(row, ['login_verifikasi_1', 'login_verifikasi_2', 'login_verifikasi_3']), axis=1)

merged_df['final_efisiensi'] = merged_df.apply(
    lambda row: get_final_sentiment(row, ['efisiensi_1', 'efisiensi_2', 'efisiensi_3']), axis=1)

merged_df['final_layanan_pengguna'] = merged_df.apply(
    lambda row: get_final_sentiment(row, ['layanan_pengguna_1', 'layanan_pengguna_2', 'layanan_pengguna_3']), axis=1)

merged_df['final_responsivitas'] = merged_df.apply(
    lambda row: get_final_sentiment(row, ['responsivitas_1', 'responsivitas_2', 'responsivitas_3']), axis=1)

merged_df.insert(5, 'final_login_verifikasi', merged_df.pop('final_login_verifikasi'))
merged_df.insert(9, 'final_efisiensi', merged_df.pop('final_efisiensi'))
merged_df.insert(13, 'final_layanan_pengguna', merged_df.pop('final_layanan_pengguna'))
merged_df.insert(17, 'final_responsivitas', merged_df.pop('final_responsivitas'))

merged_df
merged_df.to_csv("final_sentiment_labeling_results.csv", index=False)

##Labeling Result Evaluation

###Krippendorff Alpha Calculation

In [ ]:
!pip install krippendorff
import krippendorff

def prepare_data_for_alpha(data, annotator_columns):
    data_subset = data[annotator_columns]
    data_alpha = data_subset.transpose().values.tolist()
    return data_alpha

login_verifikasi_data = prepare_data_for_alpha(
    merged_df,
    ['login_verifikasi_1', 'login_verifikasi_2', 'login_verifikasi_3']
)
efisiensi_data = prepare_data_for_alpha(
    merged_df,
    ['efisiensi_1', 'efisiensi_2', 'efisiensi_3']
)
layanan_pengguna_data = prepare_data_for_alpha(
    merged_df,
    ['layanan_pengguna_1', 'layanan_pengguna_2', 'layanan_pengguna_3']
)
responsivitas_data = prepare_data_for_alpha(
    merged_df,
    ['responsivitas_1', 'responsivitas_2', 'responsivitas_3']
)

alpha_login_verifikasi = krippendorff.alpha(reliability_data=login_verifikasi_data, level_of_measurement='nominal')
alpha_efisiensi = krippendorff.alpha(reliability_data=efisiensi_data, level_of_measurement='nominal')
alpha_layanan_pengguna = krippendorff.alpha(reliability_data=layanan_pengguna_data, level_of_measurement='nominal')
alpha_responsivitas = krippendorff.alpha(reliability_data=responsivitas_data, level_of_measurement='nominal')

###Imbalance Ratio Calculation

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/drive/MyDrive/Data_Skripsi/final_sentiment_labeling_results.csv')
data = data[['final_login_verifikasi', 'final_efisiensi', 'final_layanan_pengguna', 'final_responsivitas']]

def calculate_imbalance_metrics(y_data):
    irlbl = []

    for column in y_data.columns:
        counts = y_data[column].value_counts()

        count_negative = counts.get(1, 0)
        count_positive = counts.get(2, 0)

        min_count = min(count_negative, count_positive)
        max_count = max(count_negative, count_positive)
        irlbl_value = max_count / (min_count + 1e-6) if min_count > 0 else np.inf
        irlbl.append(irlbl_value)

    irlbl = np.array(irlbl)

    # MeanIR, MaxIR, CVIR
    mean_ir = np.mean(irlbl)
    max_ir = np.max(irlbl)
    cvir = np.std(irlbl) / mean_ir if mean_ir > 0 else np.inf

    return {
        "IRLbl": irlbl,
        "MeanIR": mean_ir,
        "MaxIR": max_ir,
        "CVIR": cvir
    }

metrics = calculate_imbalance_metrics(data)

print("Imbalance Metrics:")
print("IRLbl per label:", metrics["IRLbl"])
print("MeanIR:", metrics["MeanIR"])
print("MaxIR:", metrics["MaxIR"])
print("CVIR:", metrics["CVIR"])


###Visualization

In [ ]:
for column in columns:
    data[column] = data[column].astype(str).replace({
        '1': "negative",
        '1.0': "negative",
        '2': "positive",
        '2.0': "positive"
    })

def clean_column(col):
    return (col.astype(str)
              .str.strip()
              .replace({'1': "negative", '2': "positive",
                       '1.0': "negative", '2.0': "positive"}))

for column in columns:
    data[column] = clean_column(data[column])

for column in columns:
    print(f"\nDistribusi nilai di {column}:")
    print(data[column].value_counts(dropna=False))

In [ ]:
positive_reviews = data[
    (data['final_login_verifikasi'] == 'positive') |
    (data['final_efisiensi'] == 'positive') |
    (data['final_layanan_pengguna'] == 'positive') |
    (data['final_responsivitas'] == 'positive')
]['review']

negative_reviews = data[
    (data['final_login_verifikasi'] == 'negative') |
    (data['final_efisiensi'] == 'negative') |
    (data['final_layanan_pengguna'] == 'negative') |
    (data['final_responsivitas'] == 'negative')
]['review']

positive_text = " ".join(review for review in positive_reviews.dropna())
negative_text = " ".join(review for review in negative_reviews.dropna())

if positive_text:
    wordcloud_positive = WordCloud(width=800, height=400, background_color="white").generate(positive_text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_positive, interpolation='bilinear')
    plt.axis("off")
    plt.title('Positive Sentiment WordCloud')
    plt.tight_layout(pad=0)
    plt.show()
else:
    print("Tidak ada ulasan positif untuk membuat WordCloud.")

if negative_text:
    wordcloud_negative = WordCloud(width=800, height=400, background_color="white").generate(negative_text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_negative, interpolation='bilinear')
    plt.axis("off")
    plt.title('Negative Sentiment WordCloud')
    plt.tight_layout(pad=0)
    plt.show()
else:
    print("Tidak ada ulasan negatif untuk membuat WordCloud.")



In [ ]:
aspect_data_filtered = data.replace('irrelevant', np.nan).replace('equal', np.nan).dropna(how='all', subset=aspect_columns)

sentiment_comparison = {}
for col, label in zip(aspect_columns, aspect_labels):
    counts = aspect_data_filtered[col].value_counts()
    sentiment_comparison[label] = {
        'Positive': counts.get('positive', 0),
        'Negative': counts.get('negative', 0),
        'Equal': data[col].value_counts().get('equal', 0)
    }

sentiment_df = pd.DataFrame(sentiment_comparison).T

fig, ax = plt.subplots(figsize=(12, 7))

sentiment_df[['Positive', 'Negative', 'Equal']].plot(kind='bar', ax=ax, color=['green', 'red', 'gray'])

plt.title('Comparison of Positive, Negative and Equal Sentiments per Aspect')
plt.xlabel('Aspects')
plt.ylabel('Number of Reviews')
plt.xticks(rotation=0)
plt.legend(title='Sentiments')

for container in ax.containers:
    for bar in container:
        height = bar.get_height()
        if height > 0:
            ax.annotate(f'{int(height)}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 5),  # Offset
                        textcoords='offset points',
                        ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()


#Data Modeling

In [ ]:
import numpy as np

data = pd.read_csv('/content/drive/MyDrive/Data_Skripsi/data_split.csv')
data

##Splitting Data

In [ ]:
!pip install scikit-multilearn
import skmultilearn
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
x = data[['review', 'stemmed_review']].values  # Fitur
y = data[['final_login_verifikasi', 'final_efisiensi',
          'final_layanan_pengguna', 'final_responsivitas']].values  # Target multilabel

x_model, y_model, x_valid, y_valid = iterative_train_test_split(
    x, y, test_size=0.1
)

print("Jumlah data pada train set (x_model):", x_model.shape)
print("Jumlah data pada train label (y_model):", y_model.shape)
print("Jumlah data pada test set (x_valid):", x_valid.shape)
print("Jumlah data pada test label (y_valid):", y_valid.shape)

###Save Validation Data

In [ ]:
data_valid = pd.DataFrame(x_valid, columns=['review', 'stemmed_review'])
labels_valid = pd.DataFrame(y_valid, columns=['final_login_verifikasi', 'final_efisiensi',
                                              'final_layanan_pengguna', 'final_responsivitas'])
data_valid = pd.concat([data_valid, labels_valid], axis=1)

data_valid.to_csv('data_validasi.csv', index=False)

print("Data validasi berhasil disimpan ke 'data_validasi.csv'")

###Save Modeling Data

In [ ]:
data_model = pd.DataFrame(x_model, columns=['review', 'stemmed_review'])
labels_model = pd.DataFrame(y_model, columns=['final_login_verifikasi', 'final_efisiensi',
                                              'final_layanan_pengguna', 'final_responsivitas'])
data_model = pd.concat([data_model, labels_model], axis=1)

data_model.to_csv('data_model.csv', index=False)

print("Data model berhasil disimpan ke 'data_model.csv'")

##Modeling

###Modeling Data Split

In [ ]:
!pip install scikit-multilearn
import skmultilearn
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np

In [ ]:
x_train_80, y_train_80, x_test_20, y_test_20 = iterative_train_test_split(x_model, y_model, test_size=0.2)

print("Jumlah data pada train set setelah splitting kedua (x_train_80):", x_train_80.shape)
print("Jumlah data pada train label setelah splitting kedua (y_train_80):", y_train_80.shape)
print("Jumlah data pada test set setelah splitting kedua (x_test_20):", x_test_20.shape)
print("Jumlah data pada test label setelah splitting kedua (y_test_20):", y_test_20.shape)

In [ ]:
x_train_70, y_train_70, x_test_30, y_test_30 = iterative_train_test_split(x_model, y_model, test_size=0.3)

print("Jumlah data pada train set setelah splitting kedua (x_train_70):", x_train_70.shape)
print("Jumlah data pada train label setelah splitting kedua (y_train_70):", y_train_70.shape)
print("Jumlah data pada test set setelah splitting kedua (x_test_30):", x_test_30.shape)
print("Jumlah data pada test label setelah splitting kedua (y_test_30):", y_test_30.shape)

###TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

x_train_80 = pd.DataFrame(x_train_80, columns=['review', 'stemmed_review'])
x_test_20 = pd.DataFrame(x_test_20, columns=['review', 'stemmed_review'])
x_train_70 = pd.DataFrame(x_train_70, columns=['review', 'stemmed_review'])
x_test_30 = pd.DataFrame(x_test_30, columns=['review', 'stemmed_review'])

tfidf_vectorizer = TfidfVectorizer()

# TF-IDF untuk split 80:20
tfidf_vectorizer_80 = TfidfVectorizer()
x_train_tfidf_80 = tfidf_vectorizer_80.fit_transform(x_train_80['stemmed_review'])
x_test_tfidf_20 = tfidf_vectorizer_80.transform(x_test_20['stemmed_review'])

# TF-IDF untuk split 70:30
tfidf_vectorizer_70 = TfidfVectorizer()
x_train_tfidf_70 = tfidf_vectorizer_70.fit_transform(x_train_70['stemmed_review'])
x_test_tfidf_30 = tfidf_vectorizer_70.transform(x_test_30['stemmed_review'])

print("Jumlah fitur TF-IDF (80:20):", len(tfidf_vectorizer_80.get_feature_names_out()))
print("Contoh fitur TF-IDF (80:20):", tfidf_vectorizer_80.get_feature_names_out()[:5])

print("Jumlah fitur TF-IDF (70:30):", len(tfidf_vectorizer_70.get_feature_names_out()))
print("Contoh fitur TF-IDF (70:30):", tfidf_vectorizer_70.get_feature_names_out()[:5])

print("Shape TF-IDF untuk x_train_tfidf_80:", x_train_tfidf_80.shape)
print("Shape TF-IDF untuk x_test_tfidf_20:", x_test_tfidf_20.shape)
print("Shape TF-IDF untuk x_train_tfidf_70:", x_train_tfidf_70.shape)
print("Shape TF-IDF untuk x_test_tfidf_30:", x_test_tfidf_30.shape)


###Resampling

####ML-ROS

In [ ]:
from scipy.sparse import vstack
from sklearn.utils import resample
import numpy as np
import pandas as pd
from sklearn.utils import resample

label_columns = ['final_login_verifikasi', 'final_efisiensi',
                 'final_layanan_pengguna', 'final_responsivitas']

y_train_80_df = pd.DataFrame(y_train_80, columns=label_columns)
y_train_70_df = pd.DataFrame(y_train_70, columns=label_columns)

def ml_ros(x_train, y_train):

    metrics = calculate_imbalance_metrics(y_train)
    irlbl = metrics["IRLbl"]
    mean_ir = metrics["MeanIR"]

    x_train_resampled = x_train.copy()
    y_train_resampled = y_train.copy()

    for label_idx, irlbl_value in enumerate(irlbl):
        if irlbl_value > mean_ir:
            negative_class = y_train.iloc[:, label_idx] == 1
            positive_class = y_train.iloc[:, label_idx] == 2

            # Jumlah sampel di setiap kelas
            count_negative = negative_class.sum()
            count_positive = positive_class.sum()

            # Kelas minoritas berdasarkan jumlah sampel
            if count_negative < count_positive:
                minority_indices = np.where(negative_class)[0]
            else:
                minority_indices = np.where(positive_class)[0]

            if len(minority_indices) == 0:
                continue

            # Jumlah sampel yang perlu ditambahkan
            n_samples_to_clone = int(len(minority_indices) * (irlbl_value - 1))
            if n_samples_to_clone <= 0: #jika tidak perlu menambahkan label
                continue

            # Resample fitur dan label
            cloned_indices = resample(minority_indices, n_samples=n_samples_to_clone, replace=True, random_state=42) #menambahkan jumlah sampel yang ingin ditambahkan
            cloned_features = x_train[cloned_indices, :]
            cloned_labels = y_train.iloc[cloned_indices, :]

            x_train_resampled = vstack([x_train_resampled, cloned_features])
            y_train_resampled = pd.concat([y_train_resampled, cloned_labels], axis=0, ignore_index=True)

    # Memastikan ukuran dataset konsisten
    assert x_train_resampled.shape[0] == y_train_resampled.shape[0], "Jumlah sampel antara x_train dan y_train tidak konsisten!" #validasi ukuran dataset

    return x_train_resampled, y_train_resampled

# Resampling untuk 80%
x_train_mlros_80, y_train_mlros_80 = ml_ros(x_train_tfidf_80, y_train_80_df)
print("Shape setelah resampling untuk 80%:", x_train_mlros_80.shape, y_train_mlros_80.shape)

# Resampling untuk 70%
x_train_mlros_70, y_train_mlros_70 = ml_ros(x_train_tfidf_70, y_train_70_df)
print("Shape setelah resampling untuk 70%:", x_train_mlros_70.shape, y_train_mlros_70.shape)


####ML-SMOTE

In [ ]:
from scipy.sparse import vstack
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

label_columns = ['final_login_verifikasi', 'final_efisiensi',
                 'final_layanan_pengguna', 'final_responsivitas']

y_train_80_df = pd.DataFrame(y_train_80, columns=label_columns)
y_train_70_df = pd.DataFrame(y_train_70, columns=label_columns)

def mlsmote(x_train, y_train, k=5):
    metrics = calculate_imbalance_metrics(y_train)
    irlbl = metrics["IRLbl"]
    mean_ir = metrics["MeanIR"]

    x_train_resampled = x_train.copy()
    y_train_resampled = y_train.copy()

    # Konversi sparse matrix menjadi dense
    x_train_dense = x_train.toarray() if hasattr(x_train, 'toarray') else x_train

    for label_idx, irlbl_value in enumerate(irlbl):
        if irlbl_value > mean_ir:
            negative_class = y_train.iloc[:, label_idx] == 1
            positive_class = y_train.iloc[:, label_idx] == 2

            count_negative = negative_class.sum()
            count_positive = positive_class.sum()

            if count_negative < count_positive:
                minority_indices = np.where(negative_class)[0]
            else:
                minority_indices = np.where(positive_class)[0]

            if len(minority_indices) == 0:
                continue

            minority_samples = x_train_dense[minority_indices]

            n_samples_to_generate = int(len(minority_indices) * (irlbl_value - 1))
            for _ in range(n_samples_to_generate):
                # Pilih sampel minoritas acak
                sample_idx = np.random.choice(minority_indices)
                sample_features = x_train_dense[sample_idx].reshape(1, -1)

                # Hitung jarak dan pilih k tetangga terdekat
                distances = pairwise_distances(sample_features, minority_samples)
                k_neighbors = min(k, len(minority_samples) - 1)
                neighbors_idx = np.argsort(distances.flatten())[:k_neighbors]

                random_neighbor_idx = np.random.choice(neighbors_idx)
                random_neighbor = minority_samples[random_neighbor_idx].reshape(1, -1)

                # Generate synthetic sample
                synthetic_sample = sample_features + (random_neighbor - sample_features) * np.random.rand()
                synthetic_sample = synthetic_sample.reshape(1, -1)

                # Label sintetis: sama dengan label kelas minoritas
                synthetic_label = y_train.iloc[sample_idx].copy()

                # Tambahkan synthetic sample ke dataset
                x_train_resampled = vstack([x_train_resampled, synthetic_sample])
                y_train_resampled = pd.concat([y_train_resampled, synthetic_label.to_frame().T], ignore_index=True)

    assert x_train_resampled.shape[0] == y_train_resampled.shape[0], "Jumlah sampel antara x_train dan y_train tidak konsisten!"

# Resampling untuk 80%
x_train_mlsmote_80, y_train_mlsmote_80 = mlsmote(x_train_tfidf_80, y_train_80_df)
print("Shape setelah resampling untuk 80%:", x_train_mlsmote_80.shape, y_train_mlsmote_80.shape)

# Resampling untuk 70%
x_train_mlsmote_70, y_train_mlsmote_70 = mlsmote(x_train_tfidf_70, y_train_70_df)
print("Shape setelah resampling untuk 70%:", x_train_mlsmote_70.shape, y_train_mlsmote_70.shape)

####REMEDIAL

In [ ]:
import numpy as np
import pandas as pd

label_columns = ['final_login_verifikasi', 'final_efisiensi',
                 'final_layanan_pengguna', 'final_responsivitas']

y_train_80_df = pd.DataFrame(y_train_80, columns=label_columns)
y_train_70_df = pd.DataFrame(y_train_70, columns=label_columns)

# Menghitung SCUMBLE
def calculate_scumble(instance, irlbl):
    minority_labels = instance * (irlbl <= np.mean(irlbl))  # Label minoritas
    majority_labels = instance * (irlbl > np.mean(irlbl))   # Label mayoritas
    scumble = np.sum(minority_labels) / (np.sum(minority_labels) + np.sum(majority_labels))
    return scumble if not np.isnan(scumble) else 0

# Mengimplementasikan algoritma REMEDIAL
def remedial(x_train, y_train):

    # Jika x_train adalah sparse matrix, ubah ke DataFrame
    if not isinstance(x_train, pd.DataFrame):
        x_train = pd.DataFrame(x_train.toarray())

    # Hitung IRLbl
    imbalance_metrics = calculate_imbalance_metrics(y_train)
    irlbl = imbalance_metrics["IRLbl"]

    # Hitung rata-rata IRLbl
    ir_mean = imbalance_metrics["MeanIR"]

    # Hitung SCUMBLE untuk setiap instance
    scumble_values = np.array([calculate_scumble(y_train.iloc[i], irlbl) for i in range(len(y_train))])
    scumble_threshold = np.mean(scumble_values)

    # Proses setiap instance
    new_data = []
    new_labels = []
    for i in range(len(x_train)):
        if scumble_values[i] > scumble_threshold:
            minority_labels = (y_train.iloc[i] * (irlbl <= ir_mean)).astype(int)
            majority_labels = (y_train.iloc[i] * (irlbl > ir_mean)).astype(int)

            # Clone instance
            new_instance = x_train.iloc[i]

            # Maintain minority and majority labels
            cloned_labels = minority_labels + majority_labels

            # Append cloned instance
            new_data.append(new_instance)
            new_labels.append(cloned_labels)

    # Gabungkan dataset asli dengan data yang baru dibuat
    if new_data:
        x_train = pd.concat([x_train, pd.DataFrame(new_data, columns=x_train.columns)], ignore_index=True)
        y_train = pd.concat([y_train, pd.DataFrame(new_labels, columns=y_train.columns)], ignore_index=True)

    return x_train, y_train

if __name__ == "__main__":

    # Dataset 80%
    x_train_remedial_80, y_train_remedial_80 = remedial(x_train_tfidf_80, y_train_80_df)
    print("Shape setelah REMEDIAL untuk 80%:", x_train_remedial_80.shape, y_train_remedial_80.shape)

    # Dataset 70%
    x_train_remedial_70, y_train_remedial_70 = remedial(x_train_tfidf_70, y_train_70_df)
    print("Shape setelah REMEDIAL untuk 70%:", x_train_remedial_70.shape, y_train_remedial_70.shape)


###Data Modeling

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, hamming_loss
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance, LabelPowerset, ClassifierChain

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import issparse, csr_matrix

def compute_hamming(y_true, y_pred):
    mlb = MultiLabelBinarizer()

    if hasattr(y_pred, "toarray"):  # Jika sparse matrix, konversi ke dense
        y_pred = y_pred.toarray()
    if hasattr(y_true, "toarray"):  # Jika sparse matrix, konversi ke dense
        y_true = y_true.toarray()

    y_true_bin = mlb.fit_transform(y_true)
    y_pred_bin = mlb.transform(y_pred)

    hamming = hamming_loss(y_true_bin, y_pred_bin)
    return hamming, 1 - hamming

# Kombinasi skenario
splits = ['80', '70']
resampling_methods = ['mlros', 'mlsmote', 'remedial']
C_values = [0.1, 1, 10]
strategies = ['normal', 'cc']

# Simpan hasil evaluasi
results = []

# x_test dan y_test sesuai rasio split
x_tests = {
    '80': x_test_tfidf_20,
    '70': x_test_tfidf_30
}
y_tests = {
    '80': y_test_20,
    '70': y_test_30
}

# Data train yang sudah di-resample sesuai kombinasi
x_trains = {
    'mlros_80': x_train_mlros_80,
    'mlros_70': x_train_mlros_70,
    'mlsmote_80': x_train_mlsmote_80,
    'mlsmote_70': x_train_mlsmote_70,
    'remedial_80': x_train_remedial_80,
    'remedial_70': x_train_remedial_70,
}
y_trains = {
    'mlros_80': y_train_mlros_80,
    'mlros_70': y_train_mlros_70,
    'mlsmote_80': y_train_mlsmote_80,
    'mlsmote_70': y_train_mlsmote_70,
    'remedial_80': y_train_remedial_80,
    'remedial_70': y_train_remedial_70,
}

# Loop semua kombinasi
for split in splits:
    for method in resampling_methods:
        for C in C_values:
            for strategy in strategies:
                key = f'{method}_{split}'
                x_train = x_trains[key]
                y_train = y_trains[key]
                x_test = x_tests[split]
                y_test = y_tests[split]

                model = LinearSVC(C=C, max_iter=10000)

                if strategy == 'normal':
                    clf = MultiOutputClassifier(model)
                elif strategy == 'cc':
                    clf = ClassifierChain(model)

                clf.fit(x_train, y_train)
                y_pred = clf.predict(x_test)

                # Hamming Loss
                hamming, acc_hamming = compute_hamming(y_test, y_pred)

                results.append({
                    'Split': split,
                    'Resampling': method,
                    'C': C,
                    'Strategy': strategy,
                    'Hamming_Loss': hamming,
                    'Hamming_Accuracy': acc_hamming
                })

result_df = pd.DataFrame(results)
display(result_df.sort_values(by='Hamming_Loss'))

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score

# Test set model paling optimal
x_test = x_test_tfidf_30
y_test = y_test_30

# Model dan strategi paling optimal
model = LinearSVC(C=1, max_iter=10000)
clf = MultiOutputClassifier(model)

# Fit model paling optimal
clf.fit(x_train_mlros_70, y_train_mlros_70)

# Prediksi
y_pred = clf.predict(x_test)

# Evaluasi per label
label_accuracies = []
print("Model Evaluation")
for i, col in enumerate(['final_login_verifikasi', 'final_efisiensi',
                         'final_layanan_pengguna', 'final_responsivitas']):
    print(f"Classification Report for {col}:")
    print(classification_report(y_test[:, i], y_pred[:, i], zero_division=0))
    accuracy = accuracy_score(y_test[:, i], y_pred[:, i])
    print(f"Accuracy for {col}: {accuracy}")
    label_accuracies.append(accuracy)
    print()

# Hitung Macro Average Accuracy
macro_avg_accuracy = sum(label_accuracies) / len(label_accuracies)
print("Macro Average Accuracy:", macro_avg_accuracy)


#Save Model

In [ ]:
import pickle
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier

# Data yang sesuai
x_train = x_trains['mlros_70']
y_train = y_trains['mlros_70']

# Vectorizer yang telah di-fit dengan data training
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(x_train_70['stemmed_review'])

# Model dengan parameter terbaik
model = LinearSVC(C=1.0, max_iter=10000)
clf = MultiOutputClassifier(model)

# Model training
clf.fit(x_train, y_train)

# Simpan model ke file pickle
with open('model_svc_mlros_70.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(tfidf_vectorizer, vec_file)

print("Model dan vectorizer berhasil disimpan!")
